In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
import pandas as pd

from sklearn import preprocessing

In [2]:
#Ignore Warnings
import warnings
warnings.filterwarnings("ignore")

In [4]:
#Extract Train and Test Data.
df_tr = pd.read_csv('Data\Intrusion-Detection\Train_data.csv',na_values='?',header=0)
df_tr.head(6)

In [5]:
df_ts = pd.read_csv('Data\Intrusion-Detection\Test_data.csv',na_values='?',header=0)
df_ts.head(6)

In [6]:
#Drop Duplicates
df_tr.drop_duplicates(keep='first')
df_ts.drop_duplicates(keep='first')

#Drop NA
df_tr.dropna()
df_ts.dropna()

#Drop ClassLabel column from input array X and Create output array y of Classlabel.
Xtr = np.array(df_tr.drop('xAttack',axis=1))
ytr = np.array(df_tr['xAttack'])
Xts = np.array(df_ts.drop('xAttack',axis=1))
yts = np.array(df_ts['xAttack'])

In [7]:
#Drop Duplicates
df_tr.drop_duplicates(keep='first')
df_ts.drop_duplicates(keep='first')

#Drop NA
df_tr.dropna()
df_ts.dropna()

#Drop ClassLabel column from input array X and Create output array y of Classlabel.
Xtr = np.array(df_tr.drop('xAttack',axis=1))
ytr = np.array(df_tr['xAttack'])
Xts = np.array(df_ts.drop('xAttack',axis=1))
yts = np.array(df_ts['xAttack'])

In [8]:
Xtr[:,1] = le.fit_transform(Xtr[:,1])
Xts[:,2] = le.fit_transform(Xts[:,2])
Xts = np.delete(Xts,0,axis=1)

ytr = le.fit_transform(ytr)
lb.fit_transform(ytr)

yts = le.fit_transform(yts)
lb.fit_transform(yts)


In [9]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

#Scale the data
scaler = StandardScaler()
scaler.fit(Xtr)
Xtr_t = scaler.transform(Xtr)
Xts_t = scaler.transform(Xts)

In [10]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

#Scale the data
scaler = StandardScaler()
scaler.fit(Xtr)
Xtr_t = scaler.transform(Xtr)
Xts_t = scaler.transform(Xts)

In [11]:
#Plot PoV
S = pca.singular_values_

lam = S**2
PoV = np.cumsum(lam)/np.sum(lam)

plt.plot(PoV)
plt.xlabel('No. of PCs')
plt.ylabel('PoV')
plt.grid()

In [12]:
#Plotting the classes, considering two features
cls1 = (ytr==0)
cls2 = (ytr==1)
cls3 = (ytr==2)
cls4 = (ytr==3)
cls5 = (ytr==4)

plt.scatter(Ztr[cls2,0],Ztr[cls2,1],color='g')
plt.scatter(Ztr[cls1,0],Ztr[cls1,1],color='r')
plt.scatter(Ztr[cls3,0],Ztr[cls3,1],color='b')
plt.scatter(Ztr[cls4,0],Ztr[cls4,1],color='y')
plt.scatter(Ztr[cls5,0],Ztr[cls5,1],color='c')
plt.grid()
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.legend(['Normal','DoS','Probe','R2L','U2R'])

In [13]:
ncomp =20
# Construct the PCA object
pca = PCA(n_components=ncomp, 
          svd_solver='randomized', whiten=True)

# Fit the PCA components on the entire dataset
pca.fit(Xtr_t)
Ztr = pca.transform(Xtr_t)
Zts = pca.transform(Xts_t)

In [14]:
logreg = LogisticRegression(multi_class='auto', solver='lbfgs')
logreg.fit(Ztr,ytr)
yhat = logreg.predict(Zts)

In [15]:
acc = np.mean(yhat == yts)
print('Accuracy:', acc)

We can use cross validation to find out the optimal number of PCs.

In [16]:
ncomp_test = np.arange(2,41)
acc = []

for icomp, ncomp in enumerate(ncomp_test):
    pca = PCA(n_components=ncomp)
    pca.fit(Xtr_t)
    Ztr = pca.transform(Xtr_t)
    Zts = pca.transform(Xts_t)
    
    logreg = LogisticRegression(multi_class='auto', solver='lbfgs')
    logreg.fit(Ztr,ytr)
    
    yhat = logreg.predict(Zts)
    acci = np.mean(yhat==yts)
    acc.append(acci)

imax = np.argmax(acc)
accuracy = acc[imax]
print(accuracy)

In [17]:
optPCA = ncomp_test[imax]
print('Optimal PCs:',optPCA)

In [18]:
ncomp =32
pca = PCA(n_components=ncomp, 
          svd_solver='randomized', whiten=True)

pca.fit(Xtr_t)
Ztr = pca.transform(Xtr_t)
Zts = pca.transform(Xts_t)

logreg = LogisticRegression(multi_class='auto', solver='lbfgs')
logreg.fit(Ztr,ytr)
yhat_tr = logreg.predict(Ztr)
acc_tr = np.mean(yhat_tr==ytr)
print('Train Accuracy:',acc_tr)

yhat_ts = logreg.predict(Zts)
acc_ts = np.mean(yhat_ts==yts)
print('Tesy Accuracy:',acc_ts)